# Analysis of the Chinook Music Store Database

The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store.

The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Overview of the Data

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Below is the schema diagram.![Schema Diagram](Diagram.png)

## Analyzing Most Popular Genres

In [4]:
%%sql
WITH track_genre AS
    (SELECT 
         t.track_id as track_id,
         g.name as genre
     FROM track t
     LEFT JOIN genre g ON g.genre_id = t.genre_id
    ),
    usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )
SELECT 
    tg.genre,
    count(usa.invoice_line_id) as tracks_sold,
    CAST(count(usa.invoice_line_id) AS FLOAT) / (SELECT COUNT(*)
                                  FROM usa_tracks_sold
                                 ) AS percentage_sold
FROM usa_tracks_sold as usa
INNER JOIN track_genre tg ON tg.track_id = usa.track_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;


 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Rock is clearly the most popular genre in the database accounting for more than 50% of sales.

## Analyzing Employee Sales Performance

In [5]:
%%sql
 WITH customer_invoice AS (
                           SELECT
                             c.support_rep_id,
                             SUM(i.total) AS total_sales
                           FROM customer c
                           LEFT JOIN invoice i on i.customer_id = c.customer_id
                           GROUP BY 1
                          )
SELECT 
    e.first_name || ' ' || e.last_name AS sales_agent,
    e.birthdate as birthdate,
    e.hire_date as hire_date,
    c.total_sales as total_sales
FROM employee e
INNER JOIN customer_invoice c ON c.support_rep_id = e.employee_id
ORDER BY 4 DESC
    

 * sqlite:///chinook.db
Done.


sales_agent,birthdate,hire_date,total_sales
Jane Peacock,1973-08-29 00:00:00,2017-04-01 00:00:00,1731.510000000004
Margaret Park,1947-09-19 00:00:00,2017-05-03 00:00:00,1584.0000000000032
Steve Johnson,1965-03-03 00:00:00,2017-10-17 00:00:00,1393.9200000000028


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

In [6]:
%%sql 
WITH country_or_other AS (SELECT
                             CASE
                                 WHEN (SELECT
                                           COUNT(*)
                                       FROM customer
                                       where country = c.country
                                      ) = 1 
                                 THEN 'Other'
                                 ELSE c.country 
                             END AS country,
                             c.customer_id,
                             i.total,
                             i.invoice_id
                        FROM customer c
                        INNER JOIN invoice i on i.customer_id = c.customer_id
                        )
SELECT 
    country,
    customers,
    total_sales,
    avg_sale_per_customer,
    avg_order_value
FROM (SELECT 
        country,
        COUNT(DISTINCT customer_id) AS customers,
        SUM(total) AS total_sales,
        SUM(total) / COUNT(DISTINCT customer_id) AS avg_sale_per_customer,
        SUM(total) / COUNT(DISTINCT invoice_id) as avg_order_value,
        CASE
            WHEN country = "Other" 
                THEN 1
                ELSE 0
        END AS sort
      FROM country_or_other
      GROUP BY 1
      ORDER BY sort ASC, total_sales DESC
     )



 * sqlite:///chinook.db
Done.


country,customers,total_sales,avg_sale_per_customer,avg_order_value
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075
France,5,389.0699999999999,77.81399999999998,7.781399999999998
Germany,4,334.62,83.655,8.161463414634147
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002
United Kingdom,3,245.52,81.84,8.768571428571429
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276
India,2,183.14999999999998,91.57499999999999,8.72142857142857
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143


Most sales are made in the USA; however, the Czech Republic has the highest average sale per customer. 

## Albums vs Individual Tracks

In [7]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.